In [1]:
#install.packages("dplyr", repos='http://cran.us.r-project.org')

In [39]:
options(jupyter.plot_mimetypes = 'image/png')

library("devtools")
library("httr")

library("caret")
library("corrplot")
library(e1071)
library(ggplot2)
library(cowplot)
library(plyr)
library(gbm)
library(reshape)
library("R.utils")
library(iptools)
library(bitops)
library(ggmap)
library(maps)
library(cshapes)
library(Imap)
library("zoo")
library("GGally")
#library("ggbiplot")
#library(ggbiplot)
library(mvoutlier)

library(dplyr)
#install.packages("iptools", repos='http://cran.us.r-project.org')


Attaching package: 'dplyr'

The following object is masked from 'package:GGally':

    nasa

The following object is masked from 'package:reshape':

    rename

The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [3]:
cumavg <- function( x, epsilon=0.75 ){
    #epsilon is the threshold above which we consider a point an outlier. Typically 0.75
    #we only consider outliers if the point is above the average (not below) since we are interested in high network traffic
    lx <- length(x)
    updatevals <- c( 2:lx )
    nmean <-1
    nmean[1] <- x[1]

    for(k in updatevals){
        nmean[k] <- (nmean[k-1] * (k-1) + x[k])/k
    }

    return(nmean)

    invisible()

}

cumsigma <- function( x, epsilon=0.75 ){
    #epsilon is the threshold above which we consider a point an outlier. Typically 0.75
    #we only consider outliers if the point is above the average (not below) since we are interested in high network traffic
    lx <- length(x)
    updatevals <- c( 2:lx )
    nvar <- nmean <-1
    nvar[1]  <- 0.0
    nmean[1] <- x[1]

    for(k in updatevals){
        nmean[k] <- (nmean[k-1] * (k-1) + x[k])/k
        nvar[k] <- ((nvar[k-1] + nmean[k-1]*nmean[k-1])*(k-1) + x[k]*x[k] )/k - nmean[k]*nmean[k]   
    }

    return(sqrt(nvar))

    invisible()

}

outliertest <- function( x, epsilon=0.75 ){
    #epsilon is the threshold above which we consider a point an outlier. Typically 0.75
    #we only consider outliers if the point is above the average (not below) since we are interested in high network traffic
    lx <- length(x)
    updatevals <- c( 2:lx )
    nvar <- nmean <- outliers <-1
    nvar[1]  <- 0.0
    nmean[1] <- x[1]
    outliers[1] <- 0 #by definition the first one is not an outlier

    for(k in updatevals){

        nmean[k] <- (nmean[k-1] * (k-1) + x[k])/k
        nvar[k] <- ((nvar[k-1] + nmean[k-1]*nmean[k-1])*(k-1) + x[k]*x[k] )/k - nmean[k]*nmean[k]
        if (k==2){
            outliers[k] <- 0
        } else {
            if (nvar[k] > 0){
             outliers[k] <- (abs(x[k] - nmean[k])/sqrt(nvar[k])) > epsilon && x[k] > nmean[k]
            }else{
             outliers[k] <- 0
            }

            if (outliers[k] > 0){
                nvar[k] <- nvar[k-1]
            }
        }
     
    }

    return(outliers)

    invisible()

}
calcnfreq <- function( x, interval=365){
    #count the number of events in the last year
    nfreq <- 1
    nfreq[1] <- 0
    npts <- length(x)
    for (k in c(2:npts)){
        nfreq[k] <- (nfreq[k-1] + 1/as.numeric(difftime(x[k],x[k-1],units='secs')))/2
    }
    return(nfreq*108000)
    invisible()
}


In [4]:
#Testing these against manual calculations
cumavg(c(1,8,654,132,4654,8,464))
cumsigma(c(1,8,654,132,4654,8,464))
outliertest(c(1,8,654,132,4654,8,464))

[1]    1.0000    4.5000  221.0000  198.7500 1089.8000  909.5000  845.8571

[1]    0.0000    3.5000  306.1906  267.9546 1798.1435 1690.2585 1572.6211

[1] 0 0 1 0 1 0 0

In [6]:
df <- read.csv('tempdata.log',stringsAsFactors=FALSE, header=FALSE, 
               col.names=c("datetime","serverip","clientip","clientport","clientlag","clientlon","sumrtt","countrtt",
                           "avgrtt","clientsub"))
df<-df[with(df, order(clientsub,datetime)), ]
df$tempcount<-1
df$dataday <- as.POSIXct(strptime(df$datetime, "%Y-%m-%dT%H:%M:%S+00:00"))
df$dataday<-format(df$dataday, "%Y%m%d")
str(df)

'data.frame':	1000 obs. of  12 variables:
 $ datetime  : chr  "2015-01-08T16:46:15+00:00" "2015-01-14T15:37:47+00:00" "2015-01-30T22:20:35+00:00" "2015-02-01T01:48:01+00:00" ...
 $ serverip  : chr  "serverip" "serverip" "serverip" "serverip" ...
 $ clientip  : chr  "OVTSW" "GKHTB" "UUNXV" "ZLDSO" ...
 $ clientport: int  23867 88927 69359 87973 80445 21236 50135 63079 11727 58854 ...
 $ clientlag : num  -21.87 38.51 79.45 -53.63 3.14 ...
 $ clientlon : num  175.13 80.11 -8.02 55.96 -146.82 ...
 $ sumrtt    : int  414 43 596 763 984 990 540 674 330 723 ...
 $ countrtt  : int  32065 28382 81681 94393 36793 61314 44812 97033 92090 72752 ...
 $ avgrtt    : num  77.5 660 137 123.7 37.4 ...
 $ clientsub : chr  "A" "A" "A" "A" ...
 $ tempcount : num  1 1 1 1 1 1 1 1 1 1 ...
 $ dataday   : chr  "20150108" "20150114" "20150130" "20150201" ...


In [8]:
df<-ddply(df,c("clientsub"),
      transform,
      ninsub = cumsum(tempcount))
df<-ddply(df,c("clientsub"),
          transform,
          avginsub = cumavg(avgrtt))
df<-ddply(df,c("clientsub"),
          transform,
          sigmainsub = cumsigma(avgrtt))
df<-ddply(df,c("clientsub"),
          transform,
          outlier = outliertest(avgrtt))
head(df[with(df, order(clientsub,datetime)), ],20)

,datetime,serverip,clientip,clientport,clientlag,clientlon,sumrtt,countrtt,avgrtt,clientsub,tempcount,dataday,ninsub,avginsub,sigmainsub,outlier
1,2015-01-08T16:46:15+00:00,serverip,OVTSW,23867,-21.869499,175.126212,414,32065,77.451691,A,1,20150108,1,77.45169,0.0000,0
2,2015-01-14T15:37:47+00:00,serverip,GKHTB,88927,38.505041,80.113544,43,28382,660.046512,A,1,20150114,2,368.74910,291.2974,0
3,2015-01-30T22:20:35+00:00,serverip,UUNXV,69359,79.445450,-8.021539,596,81681,137.048658,A,1,20150130,3,291.51562,261.7240,0
4,2015-02-01T01:48:01+00:00,serverip,ZLDSO,87973,-53.627307,55.957162,763,94393,123.712975,A,1,20150201,4,249.56496,238.0214,0
5,2015-02-25T11:40:34+00:00,serverip,ANMFV,80445,3.140436,-146.821644,984,36793,37.391260,A,1,20150225,5,207.13022,229.1859,0
6,2015-03-02T06:07:03+00:00,serverip,RGIUD,21236,-11.084194,-123.120849,990,61314,61.933333,A,1,20150302,6,182.93074,216.1016,0
7,2015-03-20T16:59:54+00:00,serverip,TYHPN,50135,67.493198,146.921460,540,44812,82.985185,A,1,20150320,7,168.65280,203.1050,0
8,2015-04-17T11:07:40+00:00,serverip,KSGWP,63079,28.727730,55.264260,674,97033,143.965875,A,1,20150417,8,165.56694,190.1627,0
9,2015-04-17T21:19:39+00:00,serverip,LMTWY,11727,64.792303,74.741397,330,92090,279.060606,A,1,20150417,9,178.17734,182.8005,0
10,2015-05-01T18:36:57+00:00,serverip,UCABS,58854,-29.166105,-85.919651,723,72752,100.625173,A,1,20150501,10,170.42213,174.9735,0


In [9]:
calcnfreq <- function( x, interval=365){
    #count the number of events in the last year
    nfreq <- 1
    nfreq[1] <- 0
    npts <- length(x)
    for (k in c(2:npts)){
        nfreq[k] <- (nfreq[k-1] + 1/as.numeric(difftime(x[k],x[k-1],units='secs')))/2
    }
    return(nfreq*108000)
    invisible()
}

In [14]:
df$asdate <- as.POSIXct(strptime(df$datetime, "%Y-%m-%dT%H:%M:%S+00:00"));
df<-ddply(df,c("clientsub"),
          transform,
          datafreq = calcnfreq(asdate))
head(df[with(df, order(clientsub,datetime)), ],20)

,datetime,serverip,clientip,clientport,clientlag,clientlon,sumrtt,countrtt,avgrtt,clientsub,tempcount,dataday,ninsub,avginsub,sigmainsub,outlier,datafreq,asdate
1,2015-01-08T16:46:15+00:00,serverip,OVTSW,23867,-21.869499,175.126212,414,32065,77.451691,A,1,20150108,1,77.45169,0.0000,0,0.00000000,2015-01-08 16:46:15
2,2015-01-14T15:37:47+00:00,serverip,GKHTB,88927,38.505041,80.113544,43,28382,660.046512,A,1,20150114,2,368.74910,291.2974,0,0.10499872,2015-01-14 15:37:47
3,2015-01-30T22:20:35+00:00,serverip,UUNXV,69359,79.445450,-8.021539,596,81681,137.048658,A,1,20150130,3,291.51562,261.7240,0,0.09089068,2015-01-30 22:20:35
4,2015-02-01T01:48:01+00:00,serverip,ZLDSO,87973,-53.627307,55.957162,763,94393,123.712975,A,1,20150201,4,249.56496,238.0214,0,0.59174969,2015-02-01 01:48:01
5,2015-02-25T11:40:34+00:00,serverip,ANMFV,80445,3.140436,-146.821644,984,36793,37.391260,A,1,20150225,5,207.13022,229.1859,0,0.32147754,2015-02-25 11:40:34
6,2015-03-02T06:07:03+00:00,serverip,RGIUD,21236,-11.084194,-123.120849,990,61314,61.933333,A,1,20150302,6,182.93074,216.1016,0,0.29181023,2015-03-02 06:07:03
7,2015-03-20T16:59:54+00:00,serverip,TYHPN,50135,67.493198,146.921460,540,44812,82.985185,A,1,20150320,7,168.65280,203.1050,0,0.17985092,2015-03-20 16:59:54
8,2015-04-17T11:07:40+00:00,serverip,KSGWP,63079,28.727730,55.264260,674,97033,143.965875,A,1,20150417,8,165.56694,190.1627,0,0.11244361,2015-04-17 11:07:40
9,2015-04-17T21:19:39+00:00,serverip,LMTWY,11727,64.792303,74.741397,330,92090,279.060606,A,1,20150417,9,178.17734,182.8005,0,1.52685009,2015-04-17 21:19:39
10,2015-05-01T18:36:57+00:00,serverip,UCABS,58854,-29.166105,-85.919651,723,72752,100.625173,A,1,20150501,10,170.42213,174.9735,0,0.80843112,2015-05-01 18:36:57


In [15]:

head(df[with(df, order(clientsub,datetime)), ],1)

,datetime,serverip,clientip,clientport,clientlag,clientlon,sumrtt,countrtt,avgrtt,clientsub,tempcount,dataday,ninsub,avginsub,sigmainsub,outlier,datafreq,asdate
1,2015-01-08T16:46:15+00:00,serverip,OVTSW,23867,-21.8695,175.1262,414,32065,77.45169,A,1,20150108,1,77.45169,0,0,0,2015-01-08 16:46:15


In [16]:
dfout <- df[,c(1,2,3,4,5,6,8,7,9,13,14,15,17,16,12,10)]
dfout<-dfout[with(dfout, order(dataday)), ]
head(dfout)

,datetime,serverip,clientip,clientport,clientlag,clientlon,countrtt,sumrtt,avgrtt,ninsub,avginsub,sigmainsub,datafreq,outlier,dataday,clientsub
382,2015-01-01T08:45:17+00:00,serverip,QHXUW,89910,12.835674,-137.29536,62347,68,916.86765,1,916.86765,0.000000,0.000000,0,20150101,K
878,2015-01-01T19:24:43+00:00,serverip,TVPSH,93285,-37.492239,10.66631,32741,725,45.16000,1,45.16000,0.000000,0.000000,0,20150101,X
268,2015-01-02T16:45:09+00:00,serverip,QAJRE,97349,9.915895,92.48581,94065,965,97.47668,1,97.47668,0.000000,0.000000,0,20150102,H
85,2015-01-03T13:24:14+00:00,serverip,YUKRO,14141,-40.119038,-64.56410,49539,818,60.56112,1,60.56112,0.000000,0.000000,0,20150103,C
86,2015-01-03T21:20:49+00:00,serverip,GHUQO,57500,-66.721504,-107.61302,31164,546,57.07692,2,58.81902,1.742101,1.888442,0,20150103,C
727,2015-01-03T04:10:50+00:00,serverip,YVJFN,57164,44.632682,-20.48990,34143,296,115.34797,1,115.34797,0.000000,0.000000,0,20150103,T


In [18]:
write.table(dfout,file='tempdatav6.csv',row.names=FALSE, col.names=FALSE,quote=FALSE,sep=",")

In [10]:
x<-c("2016-06-01T08:57:03+00:00","2016-06-05T09:25:45+00:00")
nfreq<-1
nfreq[1]<-0.7208312
k<-2
nfreq[k] <- (nfreq[k-1] + 1/as.numeric(difftime(x[k],x[k-1],units='secs'))*108000)/2
nfreq

[1] 0.7208312 0.5166656

In [12]:
nvar<-c(255.08974,0.0)
nmean<-c(180.27486,175.99467)
k<-2
x<-c(213.38863,21.908058)
sqrt(((nvar[k-1]*nvar[k-1] + nmean[k-1]*nmean[k-1])*(36) + x[k]*x[k] )/37 - nmean[k]*nmean[k]  )


[1] 252.9261